In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTEENN

In [ ]:
#data = np.loadtxt('flightdata_Jan_cleaned.csv', delimiter=',')
data = pd.read_csv('flightdata_Jan_cleaned.csv')

In [ ]:
data = data.to_numpy()

In [ ]:
np.random.shuffle(data)
X = data[:,:-1]
y = data[:,-1]

In [ ]:
y=y.astype('int')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

smoteenn = SMOTEENN(random_state=0)
X, y = smoteenn.fit_sample(X_train, y_train)

In [ ]:
BATCH_SIZE = 64
EPOCHS = 80

earlystopping = EarlyStopping(monitor='val_loss', verbose=1, patience=10)

model = Sequential()
model.add(Dense(16, input_shape=(X_train.shape[1]), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='Sigmoid'))

opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
           
# Compile model
model.compile(loss='binary_crossentropy',
               optimizer=opt,
               metrics=['accuracy']
               )
history = model.fit(X_train, y_train,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(X_test, y_test), callbacks=[earlystopping])

In [ ]:
# evaluate the model
_, train_acc = model.evaluate(X_train, y_train, verbose=0)
_, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()